In [ ]:
import speech_recognition as sr
from gtts import gTTS
import playsound
import datetime
import warnings
import calendar
import random
import wikipedia
import selenium
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer
import urllib.request, urllib.parse, urllib.error
import ssl
from bs4 import BeautifulSoup
import os
import sys
import time

In [ ]:
#ignore warnings
warnings.filterwarnings('ignore')

In [ ]:
#record audio and return as a string
def recordAudio():
    
    r = sr.Recognizer() #audio recognizer object
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source,duration=10)
        r.dynamic_energy_threshold = True 
        print('Say Something')
        audio = r.listen(source)
        
    data = ''
    
    try:
        data = r.recognize_google(audio) #googles speech recognizer
        print('You said:'+data.lower())
        return data.lower()
    except sr.UnknownValueError:
        assistantResponse('Could not understand audio')
        return 0
    except sr.RequestError as e:
        print('requesting results from google speech recogntion service error'+e)
        return 0

In [ ]:
#get virtual assistant response
num = 1
def assistantResponse(text):
    global num
    num += 1
    print(text)
    
    myobj = gTTS(text=text, lang="en", slow=False)
    file = str(num)+".mp3"
    myobj.save(file)
    playsound.playsound(file, True)  
    os.remove(file)

In [ ]:
#function for wakeword
def wakeWord(text):
    WAKE_WORDS = ['computer']
    text = str(text)
    #if text!=' ':
        #text = text.lower() #converting text to all lowercase words
    
        #check to see if users command/text contains a wakeword/phrase
    for phrase in WAKE_WORDS:
        if phrase in text:
            return True

    return False

In [ ]:
#give current date
def getDate():
    
    now = datetime.datetime.now()
    my_date = datetime.datetime.today()
    
    weekday = calendar.day_name[my_date.weekday()]
    monthNum = now.month
    dayNum = now.day
    
    month_names = ['January', 'February', 'March', 'April', 'May', 'June', 'July', 'August', 'September',
                  'October', 'November', 'December']
    ordinalNumbers = ['1st','2nd','3rd','4th','5th','6th','7th','8th','9th','10th','11th','12th','13th','14th','15th',
                     '16th','17th','18th','19th','20th','21st','22nd','23rd','24th','25th','26th','27th','28th','29th',
                     '30th','31st']
    return 'Today is '+weekday+' '+month_names[monthNum - 1]+' the '+ordinalNumbers[dayNum-1]+'.'

In [ ]:
#return random greeting response
def greeting(text):
    
    GREETING_INPUTS = ['hi','hey','hola','wassup','waddup','hello','computer']
    GREETING_RESPONSES = ['howdy','whats good','hello','hey there','how you doin?']
    if text!=' ':
        for word in text.split():
            if word in GREETING_INPUTS:
                return random.choice(GREETING_RESPONSES) + '.'
        
    return ''

#get first and last name
def getPerson(text):
    
    wordList = text.split()
    
    for i in range(len(wordList)):
        if wordList[i] == 'who' and wordList[i+1] == 'is':
            if wordList[i+2]:
                return wordList[i+2:]

In [ ]:
ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

url_home = "https://www.google.%(tld)s/"
url_search_num = "https://www.google.%(tld)s/search?hl=%(lang)s&q=%(query)s&" \
                 "num=%(num)d&btnG=Google+Search&tbs=%(tbs)s&safe=%(safe)s&" \
                 "cr=%(country)s"

#if sys.version_info[0] > 2:
 #   from http.cookiejar import LWPCookieJar
  #  from urllib.request import Request, urlopen
   # from urllib.parse import quote_plus, urlparse, parse_qs
#else:
 #   from cookielib import LWPCookieJar
  #  from urllib import quote_plus
   # from urllib2 import Request, urlopen
    #from urlparse import urlparse, parse_qs

home_folder = os.getenv('HOME')
if not home_folder:
    home_folder = os.getenv('USERHOME')
    if not home_folder:
        home_folder = '.'   # Use the current folder on error.
#cookie_jar = LWPCookieJar(os.path.join(home_folder, '.google-cookie'))
#try:
 #   cookie_jar.load()
#except Exception:
 #   pass


USER_AGENT = 'Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0)'

In [ ]:
def get_page(url, user_agent=None):
                if user_agent is None:
                    user_agent = USER_AGENT
                    request = urllib.request.Request(url)
                    request.add_header('User-Agent', user_agent)
              #      cookie_jar.add_cookie_header(request)
                    response = urllib.request.urlopen(request)
               #     cookie_jar.extract_cookies(response, request)
                    html = response.read()
                    response.close()
                #    try:
                 #       cookie_jar.save()
                  #  except Exception:
                   #     pass
                    return html

In [ ]:
def search(query, tld='com', lang='en', tbs='0', safe='off', num=10, start=0,
                       stop=None, pause=2.0, country='', extra_params=None,
                       user_agent=None, verify_ssl=True):
                query = urllib.parse.quote_plus(query)     #converts the unicode to utf-8

                if not extra_params:
                    extra_params = {}

                # Grab the cookie from the home page.
                ###get_page(url_home % vars(), user_agent)   # not needed

                # Prepare the URL of the first request.
                url = url_search_num % vars()          ##vars to stuff the values for url

                    # Sleep between requests.
                    # Keeps Google from banning you for making too many requests.
                time.sleep(pause)

                    # Request the Google Search results page.
                html = get_page(url, user_agent)

                soup = BeautifulSoup(html, 'html.parser')
                #print(soup.prettify())
                #print("###################################################")
                anchors = soup.find('div',attrs={"class":"uozReb"})  ##bulleted list class="TYvcTb"
                if anchors==None:
                    print("No Description in the webpage")
                    exit()
                anchors2=anchors.find("span")    ##findall(span) gives duplicate values so used find
                #print(anchors2.text)   ## was not able to get the text while using for loop for anchors2
                assistantResponse(anchors2.text)

                anchors3 = soup.find('ul', attrs={"class": "TYvcTb"})    ##bulleted list class="TYvcTb"
                if anchors3 == None:
                    print("No bulleted list")
                else:
                    assistantResponse(anchors3.text)##findall(span) gives duplicate values so used find

In [ ]:
def search_web(input): 
    input = input.lower()
    chrome_options = Options()
    chrome_options.add_experimental_option("detach", True)
    path = r"C:\Windows\system32\chromedriver.exe"
    driver = webdriver.Chrome(path, chrome_options=chrome_options)
    
    if 'play' in input and 'youtube' in input:
        indx = input.split().index('play') 
        query = input.split()[indx + 1:] 
        driver.get("http://www.youtube.com/results?search_query="+"+".join(query))
        results = driver.find_elements_by_xpath('//*[@id="video-title"]/yt-formatted-string')  #finds webresults
        results[0].click()
        
    elif 'youtube' in input: 
        assistantResponse("Opening in youtube") 
        indx = input.split().index('youtube') 
        query = input.split()[indx + 1:] 
        driver.get("http://www.youtube.com/results?search_query="+"+".join(query))   
  
    else: 
        assistantResponse("Searching in google")
        if 'google' in input or 'search' in input: 
            if 'search' in input:
                indx = input.split().index('search')
            else:
                indx = input.split().index('google')
            query = input.split()[indx + 1:]
            driver.get("https://www.google.com/search?q="+"+".join(query))
            #res = driver.find_element_by_xpath('//div[@class="r"]/a/h3')
            #ans = res.text
            #print(ans)
            search(" ".join(query), tld="com", num=10, stop=10, pause=2)

            ## doubts : 1.dont know how many searches it might allow
            ##          2.whether the coolies will get downloaded in the new system
            
        else: 
  
            driver.get("https://www.google.com/search?q="+input.split()) 
  
        return driver
    

In [ ]:
while True:
    text = recordAudio()
    response = ''
    
    if(wakeWord(text)==True):
        response = response + greeting(text)
        
        if "who are you" in text or "define yourself" in text: 
            response = response + "Hello, I am phoenix. Your personal Assistant.I am here to make your life easier."
             
        if "who made you" in text or "who created you" in text: 
            response = response+ "I have been created by sexmaxx Lavanya."
            
        if('date' in text):
            get_date = getDate()
            response = response + ' ' + get_date
            
        if ('time' in text):
            now = datetime.datetime.now()
            meridim = ''
            if now.hour>=12:
                meridim = 'pm'
                hour = now.hour - 12
            else:
                meridim = 'am'
                hour = now.hour
            
            if now.minute<10:
                minute = '0'+str(now.minute)
            else:
                minute = str(now.minute)
                
            response = response+ ' '+'It is'+ str(hour)+ ':' + minute+ ' ' +meridim+ '.'
            
        if('who is' in text):
            person = getPerson(text)
            wiki = wikipedia.summary(person, sentences = 2)
            response = response+ ' ' +wiki
            
        if 'youtube' in text or 'google' in text or 'search' in text:
            search_web(text)
            

    if response=='':
        assistantResponse('speak again')
        continue
    else:
        assistantResponse(response)       